In [2]:
#=======================================================================================================================================
# 허깅페이스 Trainer를 이용하여 NSP+MLM 훈련시키기
#
# => load_dataset 으로 wiki 연속된 문장이 있는 말뭉치를 로딩하고, 이를 토크화 시키고, 
# 연속된 문장인지 아닌지 NSP 문장을 만들고, (Lable, next_sentence_label 필드 추가)
# 해당 문장 input_ids 에 대해 15% 확률로 [MASK]를 씌워서, 실제 모델을 훈련시키는 예제 
#
# => MLM 훈련 말뭉치는 pre-kowiki-20220620-2줄.txt 사용, 평가 말뭉치는 bongsoo/bongevalsmall 사용
#
# 출처 : https://wikidocs.net/166817
#=======================================================================================================================================

import torch
import os

from tqdm.notebook import tqdm
from transformers import AutoTokenizer, BertTokenizerFast, BertConfig, BertForMaskedLM

import sys
sys.path.append("..")
from myutils import GPU_info, seed_everything, mlogging

# wand 비활성화 
# => trainer 로 훈련시키면 기본이 wandb 활성화이므로, 비활성화 시킴
os.environ["WANDB_DISABLED"] = "true"

In [3]:
# 훈련시킬 말뭉치
# => 한줄에 연속된 문장이 있어야함.
# -> .으로 구분된 한줄 문자이 아니라. 한줄에 .로구분된 여러문장이 이어진 문장이어야 함
# -> 예시:'제임스 얼 "지미" 카터 주니어는 민주당 출신 미국 39번째 대통령 이다.지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.
# => 사전 만들때 동일한 말뭉치 이용.
input_corpus = "../../data11/korpora/kowiki_2022/pre-kowiki-20220620-2줄.txt"

# eval 말뭉치 
eval_corpus = "bongsoo/bongevalsmall"

# 기존 사전훈련된 모델
model_path = "../../data11/model/bert/mbertV2.0"  #bert-base-multilingual-cased, ../../data11/model/bert/mbertV2.0

# 기존 사전 + 추가된 사전 파일
vocab_path="../../data11/model/bert/mbertV2.0"

# 출력
OUTPATH = '../../data11/model/bert/mbertV2.0-temp/'

############################################################################
# tokenizer 관련 hyper parameter 설정
############################################################################
batch_size = 64       # batch_size
token_max_len = 128   # token_seq_len
epoch = 16            # epoch
lr = 1e-4             # learning rate(기본:5e-5)
seed = 333
############################################################################

#logging 설정
logger =  mlogging(loggername="bert-MLM-Trainer", logfilename="../../log/bert-MLM-Trainer")

device = GPU_info()
print(device)

#seed 설정
seed_everything(seed)


logfilepath:../../log/bert-MLM-Trainer_2022-11-08.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30
cuda:0


In [ ]:
# tokeinzier 생성
# => BertTokenizer, BertTokenizerFast 둘중 사용하면됨. 아니면 AutoTokenizer 사용
tokenizer = BertTokenizerFast.from_pretrained(vocab_path, max_len=token_max_len, do_lower_case=False)

# fast 토크너나이즈인지 확인
print(f'*{vocab_path} is_fast:{tokenizer.is_fast}')
print('*tokenizer_len:{}, special_token_size: {}, *tokenizer.vocab_size: {}'.format(len(tokenizer), len(tokenizer.all_special_tokens), tokenizer.vocab_size))

# 모델 로딩 further pre-training 
model = BertForMaskedLM.from_pretrained(model_path, from_tf=bool(".ckpt" in model_path)) 

#################################################################################
# 모델 embedding 사이즈를 tokenizer 크기 만큼 재 설정함.
# 재설정하지 않으면, 다음과 같은 에러 발생함
# CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)` CUDA 에러가 발생함
#  indexSelectLargeIndex: block: [306,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
#
#     해당 오류는 기존 Embedding(8002, 768, padding_idx=1) 처럼 입력 vocab 사이즈가 8002인데,
#     0~8001 사이를 초과하는 word idx 값이 들어가면 에러 발생함.
#################################################################################
model.resize_token_embeddings(len(tokenizer))

model.to(device)

In [5]:
#==================================================================================================
# load_dataset을 이용하여, 훈련/평가 dataset 로딩.
#
# [로컬 데이터 파일 로딩]
# => dataset = load_dataset("text", data_files='로컬.txt')       # text 로컬 파일 로딩
# => dataset = load_dataset("csv", data_files='로컬.csv')        # csv 로컬 파일 로딩
# => dataset = load_dataset("csv", data_files='로컬.tsv', delimiter="\t")  # tsv 로컬 파일 로딩
# => dataset = load_dataset("json", data_files='로컬.json')      # json 로컬 파일 로딩
# => dataset = load_dataset("pandas", data_files='로컬.pkl')     # pickled dataframe 로컬 파일 로딩
#
# [원격 데이터 파일 로딩]
# url = "https://github.com/crux82/squad-it/raw/master/"
# data_files = {
#    "train": url + "SQuAD_it-train.json.gz",
#    "test": url + "SQuAD_it-test.json.gz",
# }
# squad_it_dataset = load_dataset("json", data_files=data_files, field="data")
#
# 출처 : https://wikidocs.net/166816
#==================================================================================================

from datasets import load_dataset

# 훈련 말뭉치 로딩
#train_dataset = load_dataset(input_corpus)
train_dataset = load_dataset("text", data_files=input_corpus) # text 로컬 파일 로딩

# 평가 말뭉치 로딩
eval_dataset = load_dataset(eval_corpus)

# train_dataset 출력해봄
print(f"train_dataset=======================================")
print(train_dataset)
print(train_dataset['train']['text'][0:3])

print(f'\r\n\r\n')

# eval_dataset 출력해봄
print(f"eval_dataset========================================")
print(eval_dataset)
print(eval_dataset['test']['text'][0:3])

Using custom data configuration default-5ca7fdc6f8a07e74
Reusing dataset text (/MOCOMSYS/.cache/huggingface/datasets/text/default-5ca7fdc6f8a07e74/0.0.0/08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration bongsoo--bongevalsmall-5b85d54295d1a976
Reusing dataset text (/MOCOMSYS/.cache/huggingface/datasets/text/bongsoo--bongevalsmall-5b85d54295d1a976/0.0.0/08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4)


  0%|          | 0/1 [00:00<?, ?it/s]

train_dataset=======================================
DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2214480
    })
})
['조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다', ' 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다', '1962년 조지아주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사를 역임했다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다']



eval_dataset========================================
DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 200
    })
})
['국토교통부 관계자는  실무기구에서는 각 업계별로 규제혁신형 플랫폼 택시를 하기 위해서는 어떤 규제를 풀어야 한다는 자기 안이 있어야 한다 고 말했다 ', '국책연구기관의 한 관계자는  위원회가 전문성과 대표성을 갖추고 본연의 장점을 최대한 살리기 위해서는 외부 감시와 통제가 보다 활성화돼야 한다 고 지적했다 ', '게임업계 관계자는  현장 수요보다 의료진 등 특정한 누군가의 이익을 위해 게임을 중독물질  질병으로 만들려 한다는 합리적 의심이 든다 고 꼬집었다 ']


In [7]:
# NSP 훈련 데이터 만들기 
# NSP 문장 만들기 
import random
import copy
from tqdm.notebook import tqdm

def tokenizer_function_nsp(examples):
    
    bag = [item for sentence in examples['text'] for item in sentence.split('.') if item != '']
    bag_size = len(bag)
   
    sentence_a = []
    sentence_b = []
    next_label = []

    count = 0
    for paragraph in examples['text']:
        count += 1
        # 하나의 문장을 읽어와서 .기준으로 나눈다.
        sentences = [sentence for sentence in paragraph.split('.') if sentence != '']
        num_sentences = len(sentences)
         
         # . 기준으로 나눈 문장이 1이상이면..
        if num_sentences > 1:
            # 문장 a 시작번지는 랜덤하게, 해당 문장 이후로 지정
            start = random.randint(0, num_sentences-2)
            # 50/50 whether is IsNextSentence or NotNextSentence
            # 0.5 이상 랜덤값이면, 연속적인 문장으로 만듬
            if random.random() >= 0.5:
                # this is IsNextSentence
                sentence_a.append(sentences[start])
                sentence_b.append(sentences[start+1])
                next_label.append(0)  #label=0이면 연속적
            # 0.5 이하 랜덤값이면  연속적이 아닌 문장으로 만듬
            else:
                index = random.randint(0, bag_size-1)
                # this is NotNextSentence
                sentence_a.append(sentences[start])
                sentence_b.append(bag[index])
                next_label.append(1)  #label=1이면 비연속적
    
    # ** return_overflowing_tokenis = False로 해야, 긴 문장인 경우 잘리더라도 다시 이어서 문장을 만들지 않는다.
    # => 입력 문장은 10개인데, 긴문장이 포함된 경우 10개를 넘는 출력이 나옴
    result = tokenizer(sentence_a, sentence_b, max_length=token_max_len, truncation=True, return_overflowing_tokens=False)
    
    # next_sentence_label next_label 복사(**deepcopy)해서 추가
    result['next_sentence_label'] = copy.deepcopy(next_label)
        
    # labels에는 inputs_id를 복사(**deepcopy)해서 추가
    result['labels'] = copy.deepcopy(result.input_ids)
     
    return result

# 훈련 NSP 데이터셋은 빠른 기본 toeknzier_function 이용하여 만듬
%time train_dataset_fast = train_dataset.map(tokenizer_function_nsp, batched=True)

  0%|          | 0/2215 [00:00<?, ?ba/s]

CPU times: user 18min 38s, sys: 18min 19s, total: 36min 57s
Wall time: 3min 47s


In [8]:
# 평가 데이터셋 처리
# 기본 tokenizer 처리
def tokenizer_function(examples):
    result =  tokenizer(examples['text'], truncation=True, max_length=token_max_len, return_overflowing_tokens=True)
    
    # 신규 인덱스와 이전 인덱스와의 매핑 추출
    # => return_overflowing_tokens=True 이면 아래 "overflow_to_sample_mapping" 해서 매핑시켜줘야함.
    sample_map = result.pop("overflow_to_sample_mapping")
    for key, values in examples.items():
        result[key] = [values[i] for i in sample_map]
    return result


# eval 평가 데이터셋은 빠른 기본 toeknzier_function 이용
%time eval_dataset_fast = eval_dataset.map(tokenizer_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

CPU times: user 176 ms, sys: 100 ms, total: 276 ms
Wall time: 109 ms


In [9]:
print(f"train_dataset_fast=======================================")
print(f'*train_len:{len(train_dataset_fast["train"])}, len:{len(train_dataset_fast["train"])}')  # fast_dataset과 dataset 길이를 비교함
print(train_dataset_fast['train'][0:2])

print(f'\r\ndecode===========================================\r\n')
print(tokenizer.decode(train_dataset_fast['train']['input_ids'][0]))
print(tokenizer.decode(train_dataset_fast['train']['input_ids'][1]))

print(f"eval_dataset_fast=======================================")
print(f'*fast_len:{len(eval_dataset_fast["test"])}, len:{len(eval_dataset["test"])}')  # fast_dataset과 dataset 길이를 비교함
print(eval_dataset_fast['test'][0:2])

print(f'\r\ndecode===========================================\r\n')
print(tokenizer.decode(eval_dataset_fast['test']['input_ids'][0]))
print(tokenizer.decode(eval_dataset_fast['test']['input_ids'][1]))

train_dataset_fast=======================================
*train_len:2214480, len:2214480
{'text': ['조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다', ' 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다'], 'input_ids': [[101, 123480, 125259, 30461, 11513, 45004, 12609, 102, 8924, 10003, 120386, 10530, 120099, 125609, 217, 124015, 217, 123984, 10459, 124436, 11467, 9641, 12609, 102], [101, 11087, 10954, 23545, 120386, 125891, 11261, 146799, 36251, 18347, 140639, 217, 136975, 33727, 8843, 118698, 25685, 9089, 10622, 9339, 17706, 102, 53519, 123914, 120238, 11513, 23969, 123670, 10530, 119570, 10739, 125056, 123198, 124013, 113, 119556, 114, 9400, 19105, 45021, 124575, 10739, 119698, 121351, 11467, 121317, 11102, 173, 118, 119577, 119550, 119588, 113, 121494, 114, 124012, 121064, 101656, 11664, 119699, 38181, 117, 124575, 123708, 119698, 121202, 10739, 60362, 89184, 120149, 12490, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 

In [10]:
# MLM을 위한 DataCollatorForLangunageModeling 호출
from transformers import DataCollatorForLanguageModeling

# input_ids에 대해 MLM 만들기
data_collator = DataCollatorForLanguageModeling(    # [MASK] 를 씌우는 것은 저희가 구현하지 않아도 됩니다! :-)
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

# input_ids MLM 만들고 출력 해봄
mlm_train_sample = data_collator(train_dataset_fast['train']['input_ids'][0:1])
mlm_eval_sample = data_collator(eval_dataset_fast['test']['input_ids'][0:1])

print(f"train_dataset_fast(MLM)=======================================")
print(train_dataset_fast)
print(mlm_train_sample['input_ids'][0])
print(train_dataset_fast['train'][0])

print(f'\r\norg===========================================\r\n')
print(tokenizer.decode(train_dataset_fast['train']['input_ids'][0]))

print(f'\r\ndecode===========================================\r\n')
print(tokenizer.decode(mlm_train_sample['input_ids'][0]))

print(f'\r\n\r\n')

print(f"eval_dataset_fast(MLM)=======================================")
print(eval_dataset_fast)
print(mlm_eval_sample['input_ids'][0])
print(eval_dataset_fast['test'][0])

print(f'\r\norg===========================================\r\n')
print(tokenizer.decode(eval_dataset_fast['test']['input_ids'][0]))

print(f'\r\ndecode===========================================\r\n')
print(tokenizer.decode(mlm_eval_sample['input_ids'][0]))

train_dataset_fast(MLM)=======================================
DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'token_type_ids', 'attention_mask', 'next_sentence_label', 'labels'],
        num_rows: 2214480
    })
})
tensor([   101, 123480, 125259,    103,  11513,  45004,  12609,    102,   8924,
         10003, 120386,  10530, 120099, 125609,    217, 124015,    103, 123984,
         10459, 124436,    103,   9641,  12609,    102])
{'text': '조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다', 'input_ids': [101, 123480, 125259, 30461, 11513, 45004, 12609, 102, 8924, 10003, 120386, 10530, 120099, 125609, 217, 124015, 217, 123984, 10459, 124436, 11467, 9641, 12609, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'next_sentence_label': 0, 'labels': [101, 123480, 125259, 30461, 11513, 45004, 12609, 102, 8924, 10003, 120386,

In [ ]:
# 훈련 trainer 설정 
# trainer 

from transformers import Trainer, TrainingArguments

#########################################################################################
# hyper parameter 설정
#########################################################################################
epochs = epoch          # epochs
#lr = 3e-5  # 학습률

total_optim_steps = len(train_dataset_fast["train"]) * epochs // batch_size   # 총 optimize(역전파) 스탭수 = 훈련dataset 계수 * epochs // 배치 크기
eval_steps=int(total_optim_steps * 0.02)           # 평가 스탭수
logging_steps=eval_steps                           # 로깅 스탭수(*평가스탭수 출력할때는 평가스탭수와 동일하게)
save_steps=int(total_optim_steps * 0.1)            # 저장 스탭수 
#save_total_limit=2                                 # 마지막 2개 남기고 삭제 

print(f'*total_optim_steps: {total_optim_steps}, *eval_steps:{eval_steps}, *logging_steps:{logging_steps}, *save_steps:{save_steps}')
#########################################################################################

# cpu 사용이면 'no_cuda = True' 설정함.
no_cuda = False
if device == 'cpu':
    no_cuda = True
print(f'*no_cuda: {no_cuda}')

training_args = TrainingArguments(
    no_cuda = no_cuda,                      # GPU 사용  안함
    output_dir = OUTPATH,                   # 출력 모델 저장 경로 
    overwrite_output_dir=True,         
    num_train_epochs=epochs,                # 에폭
    learning_rate=lr,                      # lr: 기본 5e-5
    per_gpu_train_batch_size=batch_size,    # 배치 사이즈 
    save_strategy="epoch",                  # 저장 전략 (no, epoch, steps 기본=steps) 
    save_steps=save_steps,                  # step 수마다 모델을 저장
    #save_total_limit=save_total_limit,      # 마지막 두 모델 빼고 과거 모델은 삭제
    evaluation_strategy="steps",            # 평가 전략 : steps
    eval_steps=eval_steps,                  # 평가할 스텝수
    logging_steps=logging_steps             # 로깅할 스탭수
)

# trainer로 훈련할때는 [mask] 처리된 input_ids 만 dataset으로 넘겨주면 됨.
train_datasets_input_ids = train_datasets['input_ids']
eval_dataset_fast_input_ids = eval_dataset_fast['test']['input_ids']

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,  #MLM(Masked Language Model)
    train_dataset=train_datasets_input_ids,   # 훈련 데이터셋
    eval_dataset=eval_dataset_fast_input_ids      # 평가 데이터셋
)

In [ ]:
# 훈련 시작
trainer.train()

In [ ]:
# 모델 저장
### 전체모델 저장
TMP_OUT_PATH = '../../data11/model/bert/mbertV2.0-test/'
os.makedirs(TMP_OUT_PATH, exist_ok=True)
#torch.save(model, OUTPATH + 'pytorch_model.bin') 
# save_pretrained 로 저장하면 config.json, pytorch_model.bin 2개의 파일이 생성됨
model.save_pretrained(TMP_OUT_PATH)

# tokeinizer 파일 저장(vocab)
VOCAB_PATH = TMP_OUT_PATH
tokenizer.save_pretrained(VOCAB_PATH)
print(f'==> save_model : {TMP_OUT_PATH}')